<a href="https://colab.research.google.com/github/revy1817/NLP_BG/blob/less_1_hw/HomeWork_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyconll sklearn_crfsuite corus razdel  spacy pyconll tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 83 kB 2.2 MB/s 
     |████████████████████████████████| 965 kB 52.0 MB/s 


In [ ]:
!python -m venv env 
# .\env\Scripts\activate.bat
!pip install deeppavlov
!python -m deeppavlov install squad_bert

!python -m deeppavlov install ner_ontonotes

In [2]:
import pyconll
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from numpy.random import seed

from corus import load_ne5
from razdel import tokenize
import spacy
# import deeppavlov
# from deeppavlov import configs, build_model

import tensorflow
from tensorflow.keras import Sequential, Model, Input
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical, plot_model

import nltk
from nltk.tag import DefaultTagger, BigramTagger, TrigramTagger, UnigramTagger

from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

In [ ]:
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

# **Задание №1:** Написать теггер на данных с русским языком

In [ ]:
# загрузим датасет на русском языке
full_train = pyconll.load_from_file('/content/drive/MyDrive/Colab Notebooks/Учеба/GeekBrains/Введение в обработку естественного языка/Less 5/ru_syntagrus-ud-dev.conllu')
full_test = pyconll.load_from_file('/content/drive/MyDrive/Colab Notebooks/Учеба/GeekBrains/Введение в обработку естественного языка/Less 5/ru_syntagrus-ud-test.conllu')

In [ ]:
fdata_train = []
for sent in full_train[:]:
    fdata_train.append([(token.form, token.upos) for token in sent])
    
fdata_test = []
for sent in full_test[:]:
    fdata_test.append([(token.form, token.upos) for token in sent])
    
fdata_sent_test = []
for sent in full_test[:]:
    fdata_sent_test.append([token.form for token in sent])

**Пункт № 1:** проверить UnigramTagger, BigramTagger, TrigramTagger и их комбмнации
 

In [ ]:
unigram_tagger = UnigramTagger(fdata_train)
display(unigram_tagger.tag(fdata_sent_test[100]), unigram_tagger.accuracy(fdata_test))

[('Вместе', 'ADV'),
 ('с', 'ADP'),
 ('тем', 'PRON'),
 (',', 'PUNCT'),
 ('доступ', 'NOUN'),
 ('в', 'ADP'),
 ('интернет', 'NOUN'),
 ('в', 'ADP'),
 ('РА', None),
 ('нельзя', 'ADV'),
 ('назвать', 'VERB'),
 ('дешевым', 'ADJ'),
 ('и', 'CCONJ'),
 ('общедоступным', None),
 ('.', 'PUNCT')]

0.7736867756615967

In [ ]:
bigram_tagger = BigramTagger(fdata_train, backoff=unigram_tagger)
display(bigram_tagger.tag(fdata_sent_test[100]), bigram_tagger.accuracy(fdata_test))

[('Вместе', 'ADV'),
 ('с', 'ADP'),
 ('тем', 'PRON'),
 (',', 'PUNCT'),
 ('доступ', 'NOUN'),
 ('в', 'ADP'),
 ('интернет', 'NOUN'),
 ('в', 'ADP'),
 ('РА', None),
 ('нельзя', 'ADV'),
 ('назвать', 'VERB'),
 ('дешевым', 'ADJ'),
 ('и', 'CCONJ'),
 ('общедоступным', None),
 ('.', 'PUNCT')]

0.7781807594199596

In [ ]:
trigram_tagger = TrigramTagger(fdata_train, backoff=bigram_tagger)
display(trigram_tagger.tag(fdata_sent_test[100]), trigram_tagger.accuracy(fdata_test))

[('Вместе', 'ADV'),
 ('с', 'ADP'),
 ('тем', 'PRON'),
 (',', 'PUNCT'),
 ('доступ', 'NOUN'),
 ('в', 'ADP'),
 ('интернет', 'NOUN'),
 ('в', 'ADP'),
 ('РА', None),
 ('нельзя', 'ADV'),
 ('назвать', 'VERB'),
 ('дешевым', 'ADJ'),
 ('и', 'CCONJ'),
 ('общедоступным', None),
 ('.', 'PUNCT')]

0.7776364177252847

In [ ]:
def backoff_tagger(train_sents, tagger_classes, backoff=None):
    for cls in tagger_classes:
        backoff = cls(train_sents, backoff=backoff)
    return backoff


backoff = DefaultTagger('NN') 
tag = backoff_tagger(fdata_train,  
                     [UnigramTagger, BigramTagger, TrigramTagger],  
                     backoff = backoff) 
  
tag.accuracy(fdata_test) 

0.7763135408161328

**Пункт № 2:** написать свой теггер как на занятии, попробовать разные векторайзеры, добавить знание не только букв но и слов


In [ ]:
train_tok = []
train_label = []
for sent in fdata_train[:]:
    for tok in sent:
        if tok[0] is not None:
          train_tok.append(tok[0])
          train_label.append('NO_TAG' if tok[1] is None else tok[1])
        
test_tok = []
test_label = []
for sent in fdata_test[:]:
    for tok in sent:
        if tok[0] is not None:
          test_tok.append(tok[0])
          test_label.append('NO_TAG' if tok[1] is None else tok[1])

In [ ]:
le = LabelEncoder()
train_enc_labels = le.fit_transform(train_label)
test_enc_labels = le.transform(test_label)

In [ ]:
hvectorizer = HashingVectorizer(ngram_range=(1, 3), analyzer='char', n_features=50)

X_train = hvectorizer.fit_transform(train_tok)
X_test = hvectorizer.transform(test_tok)

In [ ]:
lr = LogisticRegression(random_state=12, max_iter=40)
lr.fit(X_train, train_enc_labels)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression(max_iter=40, random_state=12)

In [ ]:
lr_pred = lr.predict(X_test)
print(classification_report(test_enc_labels, lr_pred, target_names=le.classes_, output_dict=False))

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

         ADJ       0.48      0.32      0.38     14471
         ADP       0.72      0.88      0.79     15062
         ADV       0.33      0.11      0.17      8085
         AUX       0.86      0.91      0.88      1518
       CCONJ       0.78      0.97      0.87      5736
         DET       0.47      0.36      0.41      4094
        INTJ       0.00      0.00      0.00        23
        NOUN       0.47      0.68      0.56     36568
      NO_TAG       0.99      1.00      0.99       194
         NUM       0.47      0.23      0.31      2528
        PART       0.59      0.57      0.58      4921
        PRON       0.46      0.46      0.46      8015
       PROPN       0.20      0.02      0.03      5883
       PUNCT       0.98      1.00      0.99     29463
       SCONJ       0.72      0.82      0.77      2992
         SYM       0.00      0.00      0.00       165
        VERB       0.55      0.46      0.50     18146
           X       0.00    

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
xg_reg = xgb.XGBClassifier(learning_rate = 0.1, max_depth = 7, alpha = 10, n_estimators = 50)
xg_reg.fit(X_train, train_enc_labels)

XGBClassifier(alpha=10, max_depth=7, n_estimators=50,
              objective='multi:softprob')

In [ ]:
pred = xg_reg.predict(X_test)
print(classification_report(test_enc_labels, pred, target_names=le.classes_, output_dict=False))

              precision    recall  f1-score   support

         ADJ       0.82      0.79      0.80     14471
         ADP       0.99      0.99      0.99     15062
         ADV       0.92      0.78      0.84      8085
         AUX       0.86      0.97      0.91      1518
       CCONJ       0.88      1.00      0.93      5736
         DET       0.84      0.73      0.78      4094
        INTJ       0.67      0.17      0.28        23
        NOUN       0.77      0.93      0.85     36568
      NO_TAG       1.00      1.00      1.00       194
         NUM       0.88      0.71      0.79      2528
        PART       0.98      0.73      0.84      4921
        PRON       0.83      0.89      0.86      8015
       PROPN       0.82      0.23      0.36      5883
       PUNCT       0.99      1.00      1.00     29463
       SCONJ       0.80      0.91      0.85      2992
         SYM       1.00      0.74      0.85       165
        VERB       0.87      0.80      0.84     18146
           X       0.23    

**Пункт №3:** сравнить все реализованные методы сделать выводы

Написанная модель с помощью xgboost получила самые высокие результат accuracy, если рассмотреть подобрнее classification_report то видно что модель в целом обучилась очень хорошо, исключения составляет только совсем редкие экземпляры классов

# **Задание №2:** Проверить насколько хорошо работает NER

**Пункт №1:** проверить NER из nltk/spacy/deeppavlov

Посмотрим на работу NLTK

In [ ]:
!wget http://www.labinform.ru/pub/named_entities/collection5.zip
!unzip collection5.zip

In [4]:
dir = './Collection5/'

In [5]:
docs = []
for rec in load_ne5(dir):
    words = []
    labels = []
    idx_ent = -1
    len_ents = len(rec.spans)
    rec_entities = sorted(rec.spans, key=lambda v: v.start)
    ent = None
    is_start = None
    for token in tokenize(rec.text):
        type_ent = 'OUT'
        if len_ents == 0:
            words.append(token.text)
            labels.append(type_ent)
            continue

        if (idx_ent == -1) or (idx_ent + 1 < len_ents and token.start > ent.stop):
            idx_ent += 1
            ent = rec_entities[idx_ent]
            is_start = True

        if (token.start >= ent.start) and (token.stop <= ent.stop):
                type_ent = ent.type
                is_start = False
        words.append(token.text)
        labels.append(type_ent)
    
    docs.append([words, labels])

In [ ]:
print(docs[1][0])
print(docs[1][1])
len(docs)

['Слащева', 'на', 'посту', 'главы', '"', 'СТС', 'Медиа', '"', 'займется', 'стратегией', 'и', 'продажами', 'С', '1', 'августа', 'гендиректором', '"', 'СТС', 'Медиа', '"', 'станет', 'президент', 'PR-агентства', '"', 'Михайлов', 'и', 'партнеры', '"', 'Юлиана', 'Слащева', '.', 'Президент', 'PR-агентства', '"', 'Михайлов', 'и', 'партнеры', '"', 'Юлиана', 'Слащева', ',', 'которая', ',', 'как', 'ожидается', ',', 'с', '1', 'августа', 'станет', 'гендиректором', '"', 'СТС', 'Медиа', '"', ',', 'в', 'новой', 'должности', 'займется', 'разработкой', 'стратегии', 'холдинга', ',', 'привлечением', 'творческих', 'сил', 'в', 'компанию', ',', 'а', 'также', 'усилением', 'активных', 'продаж', ',', 'сообщил', 'РИА', 'Новости', 'источник', ',', 'знакомый', 'с', 'ситуацией', '.', 'Газета', '"', 'Ведомости', '"', 'в', 'понедельник', 'со', 'ссылкой', 'на', 'источник', ',', 'близкий', 'к', 'акционерам', 'компании', ',', 'сообщила', 'о', 'возможной', 'отставке', 'гендиректора', '"', 'СТС', 'Медиа', '"', 'Бориса', 

1000

In [ ]:
for chunk in nltk.ne_chunk(nltk.pos_tag(docs[1][0])):
      if hasattr(chunk, 'label'):
         print(chunk)

(PERSON Слащева/JJ)
(PERSON Михайлов/NNP)
(PERSON Юлиана/NNP Слащева/NNP)
(PERSON Михайлов/NNP)
(PERSON Юлиана/NNP Слащева/NNP)
(PERSON Новости/NNP)
(PERSON Бориса/NNP Подольского/NNP)
(ORGANIZATION РИА/NNP Новости/NNP)
(PERSON Слащева/NNP)
(PERSON Компанией/JJ)
(ORGANIZATION Слащевой/NNP)
(ORGANIZATION СТС/JJ)
(PERSON Наверное/NN)
(PERSON Новости/JJ)
(PERSON Не/JJ)


NTLK отработал средне, СТС помечена как организация, что очень хорошо, а вот наверное, новости и не, как персоны, что очень плохо

Посмотрим на работу spacy

In [ ]:
!pip install spacy

In [ ]:
!python -m spacy download ru_core_news_sm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 15.3 MB 13.7 MB/s 
     |████████████████████████████████| 55 kB 3.3 MB/s 
     |████████████████████████████████| 8.2 MB 3.6 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')


In [ ]:
text = []
for doc in load_ne5(dir):
  text.append(doc.text)

In [ ]:
import spacy
from spacy.lang.ru.examples import sentences 

nlp = spacy.load("ru_core_news_sm")

In [ ]:
for ent in nlp(text[1]).ents:
  print(ent.text.strip(), ent.label_)

Слащева PER
СТС Медиа ORG
СТС Медиа ORG
PR-агентства ORG
Михайлов и партнеры ORG
Юлиана Слащева PER
PR-агентства ORG
Михайлов и партнеры ORG
Юлиана Слащева PER
СТС Медиа ORG
РИА Новости ORG
Ведомости ORG
СТС Медиа ORG
Бориса Подольского PER
Слащева PER
РИА Новости ORG
Слащева PER
Слащевой PER
СТС Медиа ORG
Подольскому PER
РИА Новости ORG
Слащевой PER
СТС Медиа ORG


Spacy отработал идеально

Попробуем модель deeppavlov
UPD: Нормально установить deeppavlov не получилось 

**Пункт №2:** написать свой нер попробовать разные подходы

In [6]:
# Распарсим данные на x и y:
words = []
labels = []
sentence = []

sen_count = 1 # Маркеровка предложений
for rec in load_ne5(dir):
    idx_ent = -1
    len_ents = len(rec.spans)
    rec_entities = sorted(rec.spans, key=lambda v: v.start)
    ent = None
    is_start = None
    for token in tokenize(rec.text):
        type_ent = 'OUT'
        if len_ents == 0:
            words.append(token.text)
            labels.append(type_ent)
            continue

        if (idx_ent == -1) or (idx_ent + 1 < len_ents and token.start > ent.stop):
            idx_ent += 1
            ent = rec_entities[idx_ent]
            is_start = True

        if (token.start >= ent.start) and (token.stop <= ent.stop):
                type_ent = ent.type
                is_start = False
        words.append(token.text)
        labels.append(type_ent)
        sentence.append(sen_count)
        if token.text == ".": # Т.к. у нас текст новости идет набором предложений, то разделем предложение по точкам
          sen_count += 1

In [7]:
df = pd.DataFrame({'Sentence': sentence,'Word': words, 'Span': labels})
df

,Sentence,Word,Span
0,1,Новый,OUT
1,1,командующий,OUT
2,1,коалицией,OUT
3,1,в,OUT
4,1,Афганистане,GEOPOLIT
...,...,...,...
265466,15091,курировал,OUT
265467,15091,сферу,OUT
265468,15091,межбюджетных,OUT
265469,15091,отношений,OUT


In [8]:
# Посмотрим на разброс таргета
df['Span'].value_counts(normalize=True)

OUT         0.825755
PER         0.079858
ORG         0.051422
LOC         0.017207
GEOPOLIT    0.016409
MEDIA       0.009349
Name: Span, dtype: float64

In [9]:
def get_dict_map(data, token_or_tag):
    tok2idx = {}
    idx2tok = {}
    
    if token_or_tag == 'token':
        vocab = list(set(data['Word'].to_list()))
    else:
        vocab = list(set(data['Span'].to_list()))
    
    idx2tok = {idx:tok for  idx, tok in enumerate(vocab)}
    tok2idx = {tok:idx for  idx, tok in enumerate(vocab)}
    return tok2idx, idx2tok


token2idx, idx2token = get_dict_map(df, 'token')
tag2idx, idx2tag = get_dict_map(df, 'tag')

In [10]:
idx2tag

{0: 'LOC', 1: 'OUT', 2: 'PER', 3: 'ORG', 4: 'GEOPOLIT', 5: 'MEDIA'}

In [11]:
df['Word_idx'] = df['Word'].map(token2idx)
df['Span_idx'] = df['Span'].map(tag2idx)
df.head()

,Sentence,Word,Span,Word_idx,Span_idx
0,1,Новый,OUT,28589,1
1,1,командующий,OUT,9246,1
2,1,коалицией,OUT,12427,1
3,1,в,OUT,32100,1
4,1,Афганистане,GEOPOLIT,21793,4


In [12]:
# Группируем
data_group = df.groupby(
['Sentence'],as_index=False
)['Word', 'Span', 'Word_idx', 'Span_idx'].agg(lambda x: list(x))
# Смотрим
data_group

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  after removing the cwd from sys.path.


,Sentence,Word,Span,Word_idx,Span_idx
0,1,"[Новый, командующий, коалицией, в, Афганистане...","[OUT, OUT, OUT, OUT, GEOPOLIT, OUT, OUT, OUT, ...","[28589, 9246, 12427, 32100, 21793, 20939, 1179...","[1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, ..."
1,2,"[Официальная, церемония, вступления, генерала,...","[OUT, OUT, OUT, OUT, OUT, OUT, OUT, OUT, OUT, ...","[21983, 28167, 20870, 19313, 32100, 11727, 344...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,3,"[Маккристал, ,, ранее, командовавший, в, Ираке...","[PER, OUT, OUT, OUT, OUT, GEOPOLIT, OUT, GEOPO...","[27330, 2556, 21124, 18695, 32100, 2645, 13740...","[2, 1, 1, 1, 1, 4, 1, 4, 1, 1, 1, 1, 1, 1, 1, ..."
3,4,"["", Мерой, эффективности, будет, служить, не, ...","[OUT, OUT, OUT, OUT, OUT, OUT, OUT, OUT, OUT, ...","[16507, 243, 9381, 24909, 15723, 31513, 9292, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,5,"[Гибель, мирных, жителей, в, ходе, операций, ,...","[OUT, OUT, OUT, OUT, OUT, OUT, OUT, OUT, OUT, ...","[21344, 1189, 30596, 32100, 9265, 12372, 2556,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, ..."
...,...,...,...,...,...
15086,15087,"[Заместителем, министра, финансов, РФ, в, перв...","[OUT, OUT, OUT, GEOPOLIT, OUT, OUT, OUT, OUT, ...","[23045, 21336, 20775, 20231, 32100, 21848, 830...","[1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
15087,15088,"[В, 2004-2005, гг, .]","[OUT, OUT, OUT, OUT]","[23652, 20032, 19792, 32167]","[1, 1, 1, 1]"
15088,15089,"[он, был, директором, департамента, межбюджетн...","[OUT, OUT, OUT, OUT, OUT, OUT, ORG, OUT]","[32381, 32213, 27062, 19103, 25809, 12776, 110...","[1, 1, 1, 1, 1, 1, 3, 1]"
15089,15090,"[С, конца, 2005, г, .]","[OUT, OUT, OUT, OUT, OUT]","[19915, 20698, 3495, 2730, 32167]","[1, 1, 1, 1, 1]"


In [13]:
def get_pad_train_test_val(data_group, data):

    #get max token and tag length
    n_token = len(list(set(data['Word'].to_list())))
    n_tag = len(list(set(data['Span'].to_list())))

    #Pad tokens (X var)    
    tokens = data_group['Word_idx'].tolist()
    maxlen = max([len(s) for s in tokens])
    print('maxlen = {}'.format(maxlen))
    pad_tokens = pad_sequences(tokens, maxlen=maxlen, dtype='int32', padding='post', value= n_token - 1)

    #Pad Tags (y var) and convert it into one hot encoding
    tags = data_group['Span_idx'].tolist()
    pad_tags = pad_sequences(tags, maxlen=maxlen, dtype='int32', padding='post', value= tag2idx["OUT"])
    n_tags = len(tag2idx)
    pad_tags = [to_categorical(i, num_classes=n_tags) for i in pad_tags]
    
    #Split train, test and validation set
    tokens_, test_tokens, tags_, test_tags = train_test_split(pad_tokens, pad_tags, test_size=0.1, train_size=0.9, random_state=2020)
    train_tokens, val_tokens, train_tags, val_tags = train_test_split(tokens_,tags_,test_size = 0.25,train_size =0.75, random_state=2020)

    print(
        'train_tokens length:', len(train_tokens),
        '\ntrain_spans length:', len(train_tags),
        '\ntest_tokens length:', len(test_tokens),
        '\ntest_spans:', len(test_tags),
        '\nval_tokens:', len(val_tokens),
        '\nval_spans:', len(val_tags),
    )
    
    return train_tokens, val_tokens, test_tokens, train_tags, val_tags, test_tags

train_tokens, val_tokens, test_tokens, train_tags, val_tags, test_tags = get_pad_train_test_val(data_group, df)

maxlen = 222
train_tokens length: 10185 
train_spans length: 10185 
test_tokens length: 1510 
test_spans: 1510 
val_tokens: 3396 
val_spans: 3396


In [14]:
input_dim = len(list(set(df['Word'].to_list()))) + 1
output_dim = 64
input_length = max([len(s) for s in data_group['Word_idx'].tolist()])
n_tags = len(tag2idx)
print('input_dim: ', input_dim, '\noutput_dim: ', output_dim, '\ninput_length: ', input_length, '\nn_tags: ', n_tags)

input_dim:  34940 
output_dim:  64 
input_length:  222 
n_tags:  6


In [15]:
def get_bilstm_lstm_model():
    model = Sequential()

    # Слой Embedding
    model.add(Embedding(input_dim=input_dim, output_dim=output_dim, input_length=input_length))

    # Слой bidirectional LSTM
    model.add(Bidirectional(LSTM(units=output_dim, return_sequences=True, dropout=0.2, recurrent_dropout=0.2), merge_mode = 'concat'))

    # Слой LSTM
    model.add(LSTM(units=output_dim, return_sequences=True, dropout=0.5, recurrent_dropout=0.5))

    # Слой timeDistributed Layer (обеспечивает выход формата many-to-many)
    model.add(TimeDistributed(Dense(n_tags, activation="LeakyReLU")))

    #Optimiser 
    # adam =  tensorflow.keras.optimizers.Adam(learning_rate=0.0005, beta_1=0.9, beta_2=0.999)

    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()
    
    return model

In [16]:
model_bilstm_lstm = get_bilstm_lstm_model()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 222, 64)           2236160   
                                                                 
 bidirectional (Bidirectiona  (None, 222, 128)         66048     
 l)                                                              
                                                                 
 lstm_1 (LSTM)               (None, 222, 64)           49408     
                                                                 
 time_distributed (TimeDistr  (None, 222, 6)           390       
 ibuted)                                                         
                                                                 
Total params: 2,352,006
Trainable params: 2,352,006
Non-trainable params: 0
_________________________________________________________________


In [17]:
def train_model(X, y, model):
    loss = list()
    for i in range(3):
        # fit model for one epoch on this sequence
        hist = model.fit(X, y, batch_size=128, verbose=1, epochs=1, validation_split=0.2)
        loss.append(hist.history['loss'][0])
    return loss

In [18]:
results = pd.DataFrame()
model_bilstm_lstm = get_bilstm_lstm_model()
plot_model(model_bilstm_lstm)
results['with_add_lstm'] = train_model(train_tokens, np.array(train_tags), model_bilstm_lstm)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 222, 64)           2236160   
                                                                 
 bidirectional_1 (Bidirectio  (None, 222, 128)         66048     
 nal)                                                            
                                                                 
 lstm_3 (LSTM)               (None, 222, 64)           49408     
                                                                 
 time_distributed_1 (TimeDis  (None, 222, 6)           390       
 tributed)                                                       
                                                                 
Total params: 2,352,006
Trainable params: 2,352,006
Non-trainable params: 0
_________________________________________________________________
64/64 [==============================] - 119

In [19]:
predict = model_bilstm_lstm.predict(test_tokens)

In [55]:
num = 105
np.argmax(predict[num], axis=1)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1])

In [56]:
np.argmax(test_tags[num], axis=1)

array([2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1])

**Вывод:** Как видно модель обучилась выставлять 1 (OUT), связано с дисбалансом классов, можно решить если уменьшить 1 класс, нагенерировать исскуственные данные не 1 класса, или может быть сводить предложения к одному усредненому вектору, но дисбаланс слишком большой